In [7]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),"..")))
from TextClassification import Augment
from TextClassification import TextPreprocess
from TextClassification import ExtractKeyword
from TextClassification import TextVector
from TextClassification import FeatureDimensionReduce
from TextClassification import FeatureCode
from TextClassification import ModelPreparation
from TextClassification import Classifier
from TextClassification import Successor
import pandas as pd 
import numpy as np
from sklearn import preprocessing
import json
from fastcore.transform import Pipeline

# 数据准备

In [16]:
data = pd.read_csv(os.getcwd()+'/data/训练集.csv')[:1000]
data['feature'] = data['评论标题'].map(str)+data['评论内容'].map(str)
data_feature = data['feature']

data_predict_origin = pd.read_csv(os.getcwd()+'/data/测试集.csv')[:1000]
data_predict_origin['feature'] = data_predict_origin['评论标题'].map(str)+data_predict_origin['评论内容'].map(str)
data_predict_feature = data_predict_origin['feature']

In [17]:
# 加载label_array
data['label'] = preprocessing.LabelEncoder().fit_transform(data['评分'])
label_array = np.array(data['label'])
# label与类别的映射关系
label_map = dict(zip(data['评分'].unique(),preprocessing.LabelEncoder().fit_transform(data['评分'].unique())))
print('label对应关系:',label_map)

label对应关系: {4.0: 3, 5.0: 4, 1.0: 0, 3.0: 2, 2.0: 1}


In [18]:
stopwords_cy = list(pd.read_csv(os.getcwd()+'/data/define_stopwords.txt',names = ['word'], 
                                sep = 'aaa',encoding = 'UTF-8',engine='python').word)

In [19]:
parameter_file = open(os.getcwd()+'/data/parameter.json', "rb")
parameter_json = json.load(parameter_file) 

In [20]:
# 构建input_data
input_data = {}
input_data['data_feature'] = data_feature
input_data['stop_words'] = stopwords_cy
input_data['label_array'] = label_array
input_data['aug_mark'] = np.array([0 for i in range(len(label_array))])
input_data['parameter'] = parameter_json
input_data['data_predict_feature'] = data_predict_feature

# pipeline

In [21]:
pip = Pipeline([TextPreprocess.typeface_transform.char_converter,
                TextPreprocess.cut_word.jieba_cut,
                TextPreprocess.chartype_filter.number_filter,
                TextVector.traditional_vector.one_hot_keras,
                ModelPreparation.dataset_partition.split_train_test,
                Classifier.deep_learning.train.rcnn])
pip_output = pip(input_data.copy())

2022-01-28 16:06:19,512 : INFO : 文体转换已完成
100%|██████████| 1000/1000 [00:00<00:00, 939794.76it/s]
2022-01-28 16:06:21,231 : INFO : jieba分词已完成
100%|██████████| 1000/1000 [00:00<00:00, 52937.66it/s]
2022-01-28 16:06:21,273 : INFO : 特定字符过滤已完成
2022-01-28 16:06:21,315 : INFO : onehot已完成
/Users/niejikai/Desktop/程序/Pycharm/TextClassification/ModelPreparation/dataset_partition.py:31: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  docvec_array_pure = docvec_array[[pure_index]]
/Users/niejikai/Desktop/程序/Pycharm/TextClassification/ModelPreparation/dataset_partition.py:32: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(s

Epoch 1/10
19/19 [==============================] - 33s 2s/step - loss: 1.2432 - sparse_categorical_accuracy: 0.5383 - val_loss: 1.2593 - val_sparse_categorical_accuracy: 0.5050
Epoch 2/10
19/19 [==============================] - 29s 2s/step - loss: 1.1311 - sparse_categorical_accuracy: 0.5683 - val_loss: 1.2515 - val_sparse_categorical_accuracy: 0.5050
Epoch 3/10
19/19 [==============================] - 33s 2s/step - loss: 1.0859 - sparse_categorical_accuracy: 0.5683 - val_loss: 1.2381 - val_sparse_categorical_accuracy: 0.5050
Epoch 4/10
19/19 [==============================] - 32s 2s/step - loss: 1.0082 - sparse_categorical_accuracy: 0.6467 - val_loss: 1.2879 - val_sparse_categorical_accuracy: 0.4250
Epoch 5/10
19/19 [==============================] - 32s 2s/step - loss: 0.7565 - sparse_categorical_accuracy: 0.7333 - val_loss: 1.4275 - val_sparse_categorical_accuracy: 0.4300
Epoch 6/10
19/19 [==============================] - 32s 2s/step - loss: 0.5637 - sparse_categorical_accuracy: 

In [22]:
pip_outputs = [pip_output]
evaluation = Successor.ModelEvaluation(f1_score='micro')
evaluation_result = evaluation.run(pip_outputs)

100%|██████████| 1/1 [00:07<00:00,  7.50s/it]


In [23]:
prediction = Successor.ModelPredict(return_type='class')
prediction_result = prediction.run(pip_output['clf_model'], pip_output['docvec_predict_array'])

In [20]:
save = Successor.ModelSave(save_path = os.getcwd()+'/save',name = 'rcnn')
save.run(pip_output['clf_model'])

pkl模型保存方案失败,已切换为keras模型保存方案


2022-01-24 00:24:51,200 : INFO : Assets written to: /Users/niejikai/Desktop/垃圾文件//rcnn/assets


模型已保存在/Users/niejikai/Desktop/垃圾文件//rcnn/
